In [ ]:
pip install requests beautifulsoup4

In [3]:
import requests
from bs4 import BeautifulSoup
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# Scrape the site
url = "https://www.vanderbilt.edu/isss/scam-safety/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Clean unwanted tags
for tag in soup(["script", "style", "header", "footer", "nav"]):
    tag.decompose()

sections = soup.find_all(["h2", "h3", "p", "li"])

docs = []
for section in sections:
    text = section.get_text(strip=True)
    if text:
        doc = Document(
            page_content=text,
            metadata={"source": url}  # Add section-specific URLs if available
        )
        docs.append(doc)

# Build and save vectorstore
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local("scam_safety")

print("✅ Vectorstore created and saved in 'scam_safety/'")

✅ Vectorstore created and saved in 'scam_safety/'
